<a href="https://colab.research.google.com/github/viniciusacosta/rent_calculator/blob/main/webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install unidecode
!apt-get update 
!apt install chromium-chromedriver

In [ ]:
# Imports
from selenium import webdriver
from selenium.webdriver.common.by import By
#from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import unidecode
from tqdm import tqdm
from datetime import datetime

In [ ]:
# Defining the driver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
estados = {       
    'SErgipe': 'Aracaju', 
    'ALagoas': 'Maceió',
    'AMazonas': 'Manaus',
    'BAhia': 'Salvador',
    'CEara': 'Fortaleza',
    'PAraiba': 'João Pessoa',
    'PErnambuco': 'Recife',
    'Rio grande do Norte': 'Natal',
    'ESpirito santo': 'Vitória',
    'Minas Gerais': 'Belo Horizonte',
    'Parana': 'Curitiba',
    'RJ': 'Rio de Janeiro',
    'Rio grande do Sul': 'Porto Alegre',
    'Sp': 'São Paulo',
    'Santa Catarina': 'Florianópolis',  
    'Distrito Federal': 'Brasília',
    'GOias': 'Goiânia'
}


In [ ]:
url_br = {}
for key, value in estados.items():
  value = unidecode.unidecode(value).replace(' ', '-').lower()
  key = unidecode.unidecode(key).replace(' ', '-').lower()
  url = f"https://www.vivareal.com.br/aluguel/{key.lower()}/{value}/apartamento_residencial/"
  url_br[value] = url

url_br

{'aracaju': 'https://www.vivareal.com.br/aluguel/sergipe/aracaju/apartamento_residencial/',
 'belo-horizonte': 'https://www.vivareal.com.br/aluguel/minas-gerais/belo-horizonte/apartamento_residencial/',
 'brasilia': 'https://www.vivareal.com.br/aluguel/distrito-federal/brasilia/apartamento_residencial/',
 'curitiba': 'https://www.vivareal.com.br/aluguel/parana/curitiba/apartamento_residencial/',
 'florianopolis': 'https://www.vivareal.com.br/aluguel/santa-catarina/florianopolis/apartamento_residencial/',
 'fortaleza': 'https://www.vivareal.com.br/aluguel/ceara/fortaleza/apartamento_residencial/',
 'goiania': 'https://www.vivareal.com.br/aluguel/goias/goiania/apartamento_residencial/',
 'joao-pessoa': 'https://www.vivareal.com.br/aluguel/paraiba/joao-pessoa/apartamento_residencial/',
 'maceio': 'https://www.vivareal.com.br/aluguel/alagoas/maceio/apartamento_residencial/',
 'manaus': 'https://www.vivareal.com.br/aluguel/amazonas/manaus/apartamento_residencial/',
 'natal': 'https://www.vi

In [ ]:
#Criando a lista que irá armazenar todos os resultados
results = []

# Iniciando o for para iterar no dicionário de urls
for bairro, url in url_br.items():
    driver.get(url)
    print(url)
    
    try:
        driver.find_element_by_class_name("cookie-notifier__cta").click()
    except:
        print("No cookies!")
    
    for i in tqdm(range(30), desc=bairro):
        main_div = driver.find_element_by_class_name("results-main__panel")
        properties = main_div.find_elements_by_class_name("js-property-card")
        paginator = driver.find_element_by_class_name("js-results-pagination")
        next_page = paginator.find_element_by_xpath('//*[@id="js-site-main"]/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/button')
        
        for i, apartment in enumerate(properties):
            
            #Capturando o endereço do imóvel
            try:
                address = apartment.find_element_by_class_name("property-card__address").text
            except:
                adress = None
            #Capturando a área do imóvel
            try:
                area = apartment.find_element_by_class_name("js-property-card-detail-area").text
            except:
                area = None
            #capturando a quantidade de quartos
            try:
                rooms = apartment.find_element_by_class_name("js-property-detail-rooms").text
            except:
                rooms = None
            #Capturando a quantidade de banheiros
            try:
                bathrooms = apartment.find_element_by_class_name("js-property-detail-bathroom").text
            except:
                bathrooms = None
            #Capturando a quantidade de vagas na garagem
            try:
                garage = apartment.find_element_by_class_name("js-property-detail-garages").text
            except:
                garage = None
            #Capturando os amenities(informações adicionais sobre o imóvel)
            try:
                amenites = apartment.find_element_by_class_name("property-card__amenities").text
            except:
                amenites = None
            #Capturando o preço do apartamento
            try:
                price = apartment.find_element_by_class_name("js-property-card-prices").text
            except:
                price = None
            #Capturando o preço do condomínio
            try:
                condo = apartment.find_element_by_class_name("js-condo-price").text
            except:
                condo = None
            #Criando um campo com a data de execução do Crawler    
            crawled_at = datetime.now().strftime("%Y-%m-%d %H:%M")
            #Criando um dicionário de resultados
            results.append({"address": address,
                            "city":bairro,
                            "area": area,
                            "rooms": rooms,
                            "bathrooms": bathrooms,
                            "garage": garage,
                            "amenites": str(amenites).replace(" ", "_").replace("\n", " ")
                            "price_rent": str(price).split("\n")[0], #Pegando valor de aluguel
                            "condo":condo,
                            "crawled_at": crawled_at})
        #Clicando na próxima página    
        try:
            next_page.click()
        except:
            print("Next page not clickable")

#Exportando em um arquivo Excel
df = pd.DataFrame(results)
#Fechando o google Chrome
driver.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  # Remove the CWD from sys.path while we load stuff.


https://www.vivareal.com.br/aluguel/sergipe/aracaju/apartamento_residencial/
No cookies!


aracaju:   0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=nam

Next page not clickable


aracaju:  43%|████▎     | 13/30 [01:26<01:55,  6.81s/it]

...

Next page not clickable


brasilia: 100%|██████████| 30/30 [04:01<00:00,  8.05s/it]

Next page not clickable


https://www.vivareal.com.br/aluguel/goias/goiania/apartamento_residencial/
No cookies!


goiania: 100%|██████████| 30/30 [03:54<00:00,  7.82s/it]


In [ ]:
df

,address,city,area,rooms,bathrooms,garage,amenites,price_rent,condo,crawled_at
0,"Rua Professor José Freitas de Andrade, 428 - C...",aracaju,99,4 Quartos,2 Banheiros,1 Vaga,Mobiliado Condomínio_fechado Aceita_animais Ar...,R$ 1.500 /mês,None,2022-06-23 21:10
1,"Rua Napoleão Dórea, 723 - Atalaia, Aracaju - SE",aracaju,70,3 Quartos,2 Banheiros,1 Vaga,Piscina Mobiliado Elevador Aceita_animais Ar-c...,R$ 220.000 /mês,R$ 450,2022-06-23 21:10
2,"Rua Maria Vasconcelos de Andrade, 1720 - Aruan...",aracaju,80,3 Quartos,2 Banheiros,1 Vaga,Piscina Churrasqueira Elevador Condomínio_fech...,R$ 750 /mês,R$ 400,2022-06-23 21:10
3,"Luzia, Aracaju - SE",aracaju,67,2 Quartos,1 Banheiro,1 Vaga,Piscina Churrasqueira Playground Salão_de_fest...,R$ 1.250 /mês,R$ 395,2022-06-23 21:10
4,"Avenida São João Batista, 638 - Ponto Novo, Ar...",aracaju,70,3 Quartos,1 Banheiro,1 Vaga,Piscina Mobiliado Churrasqueira Condomínio_fec...,R$ 1.200 /mês,R$ 380,2022-06-23 21:10
...,...,...,...,...,...,...,...,...,...,...
17238,"Rua 9, 544 - Setor Oeste, Goiânia - GO",goiania,None,None,None,None,None,None,None,2022-06-23 22:21
17239,"Rua 9, 544 - Setor Oeste, Goiânia - GO",goiania,None,None,None,None,None,None,None,2022-06-23 22:21
17240,"Rua 9, 544 - Setor Oeste, Goiânia - GO",goiania,None,None,None,None,None,None,None,2022-06-23 22:21
17241,"Rua 9, 544 - Setor Oeste, Goiânia - GO",goiania,None,None,None,None,None,None,None,2022-06-23 22:21


In [ ]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/MyDrive/data_science/capitais_aluguel_venda.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  df.to_csv(f)

Mounted at /content/drive
